In [1]:
import torch
from torch import nn
import numpy as np
import time
from collections import Counter
from sklearn.model_selection import train_test_split

Задание 0 (0 баллов, но сделать нужно)
Проставьте знаки неравенств, исходя из вашего опыта:
𝑃(раму|мама,мыла)∗𝑃(папу|мама,мыла) 
𝑃(столу|дорога,ложка,к)∗𝑃(обеду|дорога,ложка,к) 
𝑃(Евпатий|меня,зовут)∗𝑃(Ваня|меня,зовут) 
𝑃(журналы|я,часто,читаю)∗𝑃(комиксы|я,часто,читаю) 
Попробуйте объяснить выбор для каждого из примеров.

*Ответ на задание 0: *
1) 𝑃(раму|мама,мыла) > 𝑃(папу|мама,мыла) В данном случае слово "рама" чаще употребляется в контексте после слов "мама, мыла" начала русской скороговорки "мама мыла раму...", мы довольно редко можем встретить рядом слова "мама мыла папу" или что-то подобное.

2) 𝑃(столу|дорога,ложка,к) < 𝑃(обеду|дорога,ложка,к) Тут похожая ситуация как с первым примером. Есть русская поговорка "хороша ложка к обеду", так мы говорим чаще, чем "дорога ложка к столу", вторая фраза не является поговоркой.

3) 𝑃(Евпатий|меня,зовут) < (Ваня|меня,зовут) Имя Ваня более распрастраненное, чем Евпатий, поэтому чаще по жизни нам будет встречаться фраза "меня зовут Евпатий".

4) 𝑃(журналы|я,часто,читаю) = 𝑃(комиксы|я,часто,читаю) В данном случае тут зависит от того человека, от чьего имени произносится/пишется данная фраза. Комиксы и журналы кажется примерно одинаково популярны, однозначно сказать какая фраза встречается чаще сложно, поэтому тут нельзя однозначно поставить знак <>, скорее они равны.

In [2]:
"""Источники: https://pytorch.org/tutorials/beginner/transformer_tutorial.html, маленькие куски кода из семинаров"""
# читаем файл
directory = '/kaggle/input/hse-ida-transformers/small_corp_for_test.txt'
file = open(directory, 'r')
data = file.readlines()
file.close()
# выводим количетво строк в файле
print(f"Total lines in text: {len(data)}")

Total lines in text: 700000


In [3]:
# какие символы входят в текст?
list_of_all_letters = []
for words in data:
  for letter in words:
    list_of_all_letters.append(letter)
collection = Counter(list_of_all_letters)
sorted(collection.items(), key=lambda item:item[1], reverse=True)

[(' ', 4380220),
 ('о', 2779376),
 ('а', 2366842),
 ('е', 2028774),
 ('т', 1985934),
 ('н', 1582468),
 ('и', 1501660),
 ('с', 1401746),
 ('в', 1130062),
 ('р', 1055739),
 ('л', 915552),
 ('д', 899550),
 ('м', 878018),
 ('у', 848660),
 ('к', 795405),
 ('п', 757949),
 ('\n', 700000),
 ('я', 555131),
 ('ь', 538119),
 ('ы', 403604),
 ('г', 402608),
 ('б', 384499),
 ('ч', 370993),
 ('з', 355889),
 ('й', 289713),
 ('ж', 249344),
 ('ш', 187256),
 ('э', 162631),
 ('ю', 149791),
 ('х', 147781),
 ('ц', 117824),
 ('ё', 76849),
 ('щ', 66697),
 ('ф', 60354),
 ('-', 43618),
 ('ъ', 5406)]

In [4]:
# сколько всего символов в исходном файле?
letter_counter = sum([elem for elem in collection.values()])
print(f"Всего символов в текстах до удаления символов переноса на новую строку и табуляции: {letter_counter}")

Всего символов в текстах до удаления символов переноса на новую строку и табуляции: 30576062


In [5]:
# убираем символы конца строки и табуляци
data = [line.rstrip("\n").rstrip("\t") for line in data]
print("Вот так сейчас выглядят наши данные: ")
print(data[:25])

Вот так сейчас выглядят наши данные: 
['добро', 'кого', 'капитан', 'нет', 'зачем', 'что происходит', 'что такое', 'рассказ', 'никому', 'ну что', 'кто', 'я укажу', 'исполняй', 'ждет', 'он думал', 'в броне', 'отец', 'быстро', 'ну жил', 'что', 'здоров', 'что с тобой', 'иортэ', 'как', 'ну как']


In [6]:
class Preprocessor:
    def __init__(self):
        self.alphabet = '_добсркгаупитнезчмфяжлйвцыэь-шхющёъ][ '
        self.token2ind = {}
        self.ind2token = {}
        for i in range(len(self.alphabet)):
            self.token2ind[self.alphabet[i]] = i
            self.ind2token[i] = self.alphabet[i]
        
    
    def preprocess(self, text, window_size):
        list_of_batches = []
        for word in text.lower():
          list_of_batches.append(self.token2ind[word])
        first_v = list_of_batches[1:] + [0] * (window_size - len(list_of_batches[1:]))
        second_v = list_of_batches + [0] * (window_size - len(list_of_batches))
        return first_v, second_v

In [7]:
# добавим '[' и ']' в наш текст
for i in range(len(data)):
  data[i] = '[' + data[i] + ']' 
# выводим посмотреть что получилось
print(data[25:31])

['[ну что ж]', '[ах да]', '[сударь]', '[утонул]', '[меня]', '[спросил он]']


In [8]:
THRESHOLD = 128
# удаляем слишком длинные фразы из наших данных
count_before = len(data)
data_batched = [phrase for phrase in data if len(phrase) <= 128]
count_after = len(data_batched)
print(f'Было удалено {count_before - count_after} фраз')
# разбиваем на тестовую и тренировочную выборки
data_train, data_test = train_test_split(data_batched, test_size= int(0.15 * len(data_batched)), random_state=42, shuffle = True)

Было удалено 16562 фраз


In [9]:
# вниз дальше верно
from sys import prefix
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, x, preproc, win_size = 128):
        self.x = x
        self.preproc = preproc
        self.win_size = win_size
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        first_vec, second_vec = preproc.preprocess(self.x[idx], self.win_size)
        first_vec = torch.tensor(first_vec)
        second_vec = torch.tensor(second_vec)
        return first_vec, second_vec

In [10]:
# создаем объект класса препроцессор
preproc = Preprocessor()
# обрабатываем датасет, получаем тестовые и тренировочные выборки уже подготовленные
train_dataset = TextDataset(data_train, preproc)
test_dataset = TextDataset(data_test, preproc)

ИСТОЧНИК: https://habr.com/ru/company/wunderfund/blog/594333/

Данный прием необхдим для корректной работы механизма внимания. Смысл маскирования символов в том, чтобы не дать модели "мочь видеть" что будет в будущем. К матрице скалярных произведений мы применяем маску, то есть отключаем элементы выше главной диагонали. Таким образом, выход зависит не от всей последовательности целиком, а только от n-1 предудыщих символов, если мы находимся в данный момент на n-ом символе.

In [11]:
import math
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [12]:
from torch.nn import TransformerEncoder, TransformerEncoderLayer
class LanguageModel(nn.Module):
    def __init__(self, vocab_size): # vocab_size - размер словаря
        super(LanguageModel, self).__init__()
        self.hidden_size = 64
        self.linear_size = 1024
        self.emb = nn.Embedding(vocab_size, self.hidden_size) # эмбеддинг
        self.pe = PositionalEncoding(d_model=self.hidden_size) 
        # энкодер
        self.transformer_encoder_layer = TransformerEncoderLayer(d_model=self.hidden_size, nhead=8) 
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers=6)
        self.decoder = nn.Sequential(
            nn.Linear(self.hidden_size, self.linear_size),
            nn.ReLU(),
            nn.Linear(self.linear_size, vocab_size)
        )
    
    def forward(self, x, src_mask):
        x = self.emb(x) # emb, then pe
        x = self.pe(x)
        x = x.transpose(1, 0)
        # print(x.shape, src_mask.shape)
        x = self.transformer_encoder(x, src_mask) # transformer encoder with mask
        x = self.decoder(x).transpose(2,1) # decoder
        return x.transpose(2, 0)
    
    def generate_square_subsequent_mask(self, sz):
        # А вот и то самое маскирование
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

In [13]:
model = LanguageModel(len('_добсркгаупитнезчмфяжлйвцыэь-шхющёъ][ '))

In [14]:
import tqdm
class Trainer:
    def __init__(self, model, train_dataset, test_dataset):
        
        self.model = model
        
        self.train_batch_size = 64
        self.test_batch_size = 64
        
        self.train_dataloader = torch.utils.data.DataLoader(train_dataset, self.train_batch_size)
        self.test_dataloader = torch.utils.data.DataLoader(test_dataset, self.test_batch_size)
        self.train_dataloader_size = len(self.train_dataloader)
        self.test_dataloader_size = len(self.test_dataloader)
        self.device = 'cuda:0'
        self.criterion = nn.CrossEntropyLoss(ignore_index=0)
        self.optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
        self.steps_to_print = 100
        
    def train_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        for batch in self.train_dataloader:
            x, y = batch
            # print(x.__getitem__(5))
            x, y = x.to(self.device), y.to(self.device)
            sequence_length = x.size(1)
            # print(sequence_length)
            # src_mask = self.model.generate_square_subsequent_mask(train_dataset.win_size).to(self.device)
            src_mask = self.model.generate_square_subsequent_mask(sequence_length).to(self.device)
            # print(x.shape, src_mask.shape)
            predicted = self.model(x, src_mask)
            loss = self.criterion(predicted, y)
            counted_loss += loss.detach().item()
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
            step += 1
            it += 1
            if step%self.steps_to_print == 0:
                result = 'Train epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.train_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
    
    def validate_one_epoch(self, epoch_number):
        step = 0
        counted_loss = 0
        current_time = time.time()
        it = 0
        for batch in self.test_dataloader:
            x, y = batch
            x, y = x.to(self.device), y.to(self.device)
            sequence_length = y.size(1)
            predicted = self.model(x, model.generate_square_subsequent_mask(sequence_length).to(self.device))
            loss = self.criterion(predicted, y)
            counted_loss += loss.item()
            step += 1
            it += 1
            # YOUR CODE HERE
            
            # реализуйте шаги для теста модели
            # помните, что данный метод уже запускается из 
            # блока with torch.no_grad(), а потому 
            # повторно его использовать не нужно
            
            ################
            
            if step%(self.steps_to_print//2) == 0:
                result = 'Validate epoch '+str(epoch_number)+' | '
                result += 'Step '+str(step)+'/'+str(self.test_dataloader_size)+' | '
                result += 'Counted loss '+str(counted_loss)+' | '
                result += 'ppl '+str(math.exp(counted_loss/it))+' | '
                result += 'time '+str(time.time() - current_time) + ' | '
                print(result)
                current_time = time.time()
                counted_loss = 0
                it = 0
        
    def train(self, number_of_epochs):
        model.to(self.device)
        for epoch in range(1, number_of_epochs+1):
            model.train()
            self.train_one_epoch(epoch)
            with torch.no_grad():
                model.eval()
                self.validate_one_epoch(epoch)
            print()

In [15]:
trainer = Trainer(model = model, train_dataset = train_dataset, test_dataset = test_dataset)
trainer.train(number_of_epochs = 1)

Train epoch 1 | Step 100/9077 | Counted loss 285.1531136035919 | ppl 17.3142720612153 | time 7.3764495849609375 | 
Train epoch 1 | Step 200/9077 | Counted loss 218.22955441474915 | ppl 8.866636671031873 | time 6.133527040481567 | 
Train epoch 1 | Step 300/9077 | Counted loss 184.04751229286194 | ppl 6.299530601533901 | time 6.089079141616821 | 
Train epoch 1 | Step 400/9077 | Counted loss 151.71138882637024 | ppl 4.559048266536202 | time 6.25527286529541 | 
Train epoch 1 | Step 500/9077 | Counted loss 108.43766486644745 | ppl 2.9575956228267817 | time 6.140430450439453 | 
Train epoch 1 | Step 600/9077 | Counted loss 75.8302121758461 | ppl 2.134648768185245 | time 6.09723424911499 | 
Train epoch 1 | Step 700/9077 | Counted loss 55.74238455295563 | ppl 1.7461683017548204 | time 6.137481212615967 | 
Train epoch 1 | Step 800/9077 | Counted loss 43.90214928984642 | ppl 1.5511886265690846 | time 6.090056896209717 | 
Train epoch 1 | Step 900/9077 | Counted loss 35.49918235838413 | ppl 1.42616

In [16]:
def generate_text(text):
    with torch.no_grad():
        model.eval()
        x = []
        device = 'cuda:0'
        for letter in text:
            x.append(preproc.token2ind[letter])
        x = torch.from_numpy(np.array(x)).to(device)
        sequence_length = len(x)
        src_mask = model.generate_square_subsequent_mask(sequence_length).to(device)
        pred = model(x, src_mask)
        ind = pred.topk(1)[1].view(-1)[-1].item()
        text += preproc.ind2token[ind]  
        if len(text) > 150 or text[-1] == "]":
            return text
        else:
            return generate_text(text)

In [25]:
generate_text('[ого иад ')

'[ого иад рррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррррууууууууурррруррррррррррррррррррррррррррррррррр'

Не хватило времеми, чтобы подобрать нормальные параметры модели и обучить на нужном числе эпох, этим объясняется такая странная генерация текста.